# Libraries

https://www.gurobi.com/documentation/9.5/refman/py_python_api_overview.html

In [14]:
# Libraries
import gurobipy as grb
from gurobipy import GRB

# Model

https://www.gurobi.com/documentation/9.5/refman/py_model.html#pythonclass:Model

In [15]:
 # Create a model
model = grb.Model("Model")

# Decision variables

https://www.gurobi.com/documentation/9.5/refman/py_model_addvar.html#pythonmethod:Model.addVar
https://www.gurobi.com/documentation/9.5/refman/py_model_addvars.html#pythonmethod:Model.addVars
https://www.gurobi.com/documentation/9.5/refman/py_model_addmvar.html#pythonmethod:Model.addMVar

In [16]:
# Add decision variables
# - One integer which will be associated to its decimal decomposition
# NB: 4 decimal digits number
M = model.addVar(vtype=GRB.INTEGER, name="M")
D = model.addVars([i for i in range(4)], vtype=GRB.INTEGER, lb=0, ub=9, name="D")
# - Two integers which will be associated to their binary decomposition
# NB: 10 binary digits number
N = model.addVars([i for i in range(1, 3)], vtype=GRB.INTEGER, name="N")
B = model.addVars([(i, j) for i in range(1, 3) for j in range(10)], vtype=GRB.BINARY, name="B")

In [17]:
B

{(1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (2, 9): <gurobi.Var *Awaiting Model Update*>}

# Constraints

https://www.gurobi.com/documentation/9.5/refman/py_model_addconstr.html#pythonmethod:Model.addConstr
https://www.gurobi.com/documentation/9.5/refman/py_model_addconstrs.html#pythonmethod:Model.addConstrs

In [4]:
# Add constraints

# - M must be equal to its decimal decomposition
model.addConstr(M == grb.quicksum(D[i]*(10**i) for i in range(4)), name=f"MVsDecimalDecomposition")

# - The sum of digits of the decimal decomposition must be equal to 12
model.addConstr(grb.quicksum(D[i] for i in range(4)) == 12, name=f"SumDigits")

# - The digits of the decimal decomposition must be increasing (reading from left to right)
# NB: strict inequalities cannot be implemented using < or >, must use <= and >= and +1
for i in range(3):
    model.addConstr(D[i+1] + 1 <= D[i], name=f"IncreasingDigits[{i}]")
# or
# model.addConstrs((D[i+1] + 1 <= D[i] for i in range(3)), name=f"IncreasingDigits")

# - N numbers must be associated to their binary decompositions
for i in range(1, 3):
    model.addConstr(N[i] == grb.quicksum(B[i, j]*(2**j) for j in range(10)), name=f"NVsBinaryDecomposition[{i}]")

# - The number of non-zero digits in the binary decompositions must be between 2 and 6 (both included)
for i in range(1, 3):
    model.addConstr(grb.quicksum(B[i, j] for j in range(10)) >= 2, name=f"NbNonZeroDigitsLB[{i}]")
    model.addConstr(grb.quicksum(B[i, j] for j in range(10)) <= 6, name=f"NbNonZeroDigitsUB[{i}]")

# - The sum of the numbers N must be equal to M
model.addConstr(N[1] + N[2] == M, name=f"Sum")

# - The second binary number N_2 must be larger or equal to the first N_1
model.addConstr(N[2] >= N[1], name=f"Order")

<gurobi.Constr *Awaiting Model Update*>

# Linear expressions

https://www.gurobi.com/documentation/9.5/refman/py_lex.html#pythonclass:LinExpr

In [5]:
# Define linear expression
total_sum = N[1] + N[2] + M

# Objective function

https://www.gurobi.com/documentation/9.5/refman/py_model_setobjective.html#pythonmethod:Model.setObjective
https://www.gurobi.com/documentation/9.5/refman/py_model_setobjectiven.html#pythonmethod:Model.setObjectiveN

In [6]:
# Add objective function
# i.e. maximize the difference N_2 - N_1
model.setObjective(N[2] - N[1], GRB.MAXIMIZE)

# Add multi-objective function
# i.e. minimize the sum M + N_1 + N_2 and then maximize the difference N_2 - N_1
# model.ModelSense = GRB.MINIMIZE
# model.setObjectiveN(total_sum, 0, 2)
# model.setObjectiveN(N[1] - N[2], 1, 1)

# Solving

https://www.gurobi.com/documentation/9.5/refman/py_model_optimize.html

In [7]:
# Update model
model.update()

In [8]:
# Display model
model.display()

Maximize
  -1.0 N[1] + N[2]
Subject To
  MVsDecimalDecomposition: M + -1.0 D[0] + -10.0 D[1] + -100.0 D[2] + -1000.0 D[3] = 0
  SumDigits: D[0] + D[1] + D[2] + D[3] = 12
  IncreasingDigits[0]: -1.0 D[0] + D[1] <= -1
  IncreasingDigits[1]: -1.0 D[1] + D[2] <= -1
  IncreasingDigits[2]: -1.0 D[2] + D[3] <= -1
NVsBinaryDecomposition[1]: N[1] + -1.0 B[1,0] + -2.0 B[1,1] + -4.0 B[1,2] + -8.0 B[1,3]
+ -16.0 B[1,4] + -32.0 B[1,5] + -64.0 B[1,6] + -128.0 B[1,7] + -256.0 B[1,8] + -512.0
 B[1,9] = 0
NVsBinaryDecomposition[2]: N[2] + -1.0 B[2,0] + -2.0 B[2,1] + -4.0 B[2,2] + -8.0 B[2,3]
+ -16.0 B[2,4] + -32.0 B[2,5] + -64.0 B[2,6] + -128.0 B[2,7] + -256.0 B[2,8] + -512.0
 B[2,9] = 0
NbNonZeroDigitsLB[1]: B[1,0] + B[1,1] + B[1,2] + B[1,3] + B[1,4] + B[1,5] + B[1,6] +
 B[1,7] + B[1,8] + B[1,9] >= 2
NbNonZeroDigitsUB[1]: B[1,0] + B[1,1] + B[1,2] + B[1,3] + B[1,4] + B[1,5] + B[1,6] +
 B[1,7] + B[1,8] + B[1,9] <= 6
NbNonZeroDigitsLB[2]: B[2,0] + B[2,1] + B[2,2] + B[2,3] + B[2,4] + B[2,5] + B[2,6] +
 B[

In [9]:
# Solving
# model.params.outputflag = 0 # Mute
# model.setParam('TimeLimit', 30) # Time limit in seconds
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 13 rows, 27 columns and 82 nonzeros
Model fingerprint: 0x206db896
Variable types: 0 continuous, 27 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 9e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolved: 11 rows, 24 columns, 76 nonzeros
Variable types: 0 continuous, 24 integer (20 binary)
Found heuristic solution: objective 89.0000000
Found heuristic solution: objective 123.0000000

Root relaxation: objective 8.169403e+02, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  816

In [10]:
# Get optimization status
print(model.Status == GRB.OPTIMAL, model.Status == GRB.TIME_LIMIT, model.Status == GRB.INFEASIBLE)

True False False


# Results

In [11]:
# Get objective value
objective_value = model.objVal
print(objective_value)

780.0


https://www.gurobi.com/documentation/9.5/refman/py_model_getvarbyname.html

In [12]:
# Get decision variables values
print(f"M={M.x}, N_1={N[1].x}, N_2={N[2].x}, B_(1,0)={B[1, 0].x}, B_(2,0)={B[2, 0].x}")
# or
# print(f"M={model.getVarByName('M').x}, \
#     N_1={model.getVarByName('N[1]').x}, N_2={model.getVarByName('N[2]').x}, \
#     B_(1,0)={model.getVarByName('B[1,0]').x}, B_(2,0)={model.getVarByName('B[2,0]').x}")

M=1236.0, N_1=228.0, N_2=1008.0, B_(1,0)=-0.0, B_(2,0)=-0.0


In [13]:
# Get linear expression values
print(total_sum.getValue())

2472.0
